In [26]:
import pandas as pd
import time

pd.options.display.max_rows = None
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 20

In [27]:
horse_vs_df = pd.read_csv('3.4.horse_vs_data.csv')
horse_vs_df = horse_vs_df.iloc[:,:28]
df = pd.read_csv('3.1.df_add_id.csv')
print(len(horse_vs_df))
print(len(df))
# time.sleep(10)

15883
1366


着順の１位　＝　NAN 走ってないうま　＝　NAN これをくべつする

## メインデータの前処理

In [28]:
def preprocessing(df):
  # 着順
  df["着順"] = df["着順"].map(lambda x:str(x).replace('除外', "21"))
  df["着順"] = df["着順"].map(lambda x:str(x).replace('取消', "21"))
  df["着順"] = df["着順"].map(lambda x:str(x).replace('中止', "21"))
  df["着順"] = df["着順"].astype(int)
  # 性齢
  df["性"] = df["性齢"].map(lambda x:str(x)[0])
  df["齢"] = df["性齢"].map(lambda x:str(x)[1:]).astype(int)
  # 人気
  df["人気"] = df["人気"].map(lambda x:str(x).replace('nan', "21"))
  df["人気"] = df["人気"].map(lambda x:str(x).replace('.0', ''))
  df["人気"] = df["人気"].astype(int)
  # タイム
  df['タイム'] = df['タイム'].fillna("9:99.9")
  # 着差
  # df.loc[df['着差'] == "-", '着差'] = "除外"
  # df["着差"] = df["着差"].fillna("１着")
  # オッズ
  df["単勝オッズ"] = df["単勝オッズ"].fillna(0)
  df["単勝オッズ"] = df["単勝オッズ"].astype(float)
  # 後3F
  df["後3F"] = df["後3F"].fillna(99.9)
  df["後3F"] = df["後3F"].astype(float)
  # コーナー通過順
  df["コーナー通過順"] = df["コーナー通過順"].fillna(0-0-0-0)
  # 馬体重
  df["馬体重(増減)"] = df["馬体重(増減)"].fillna("999(999)")
  df["体重"] = df["馬体重(増減)"].str.split('(',expand=True)[0].astype(int)
  df["体重変化"] = df["馬体重(増減)"].str.split('(',expand=True)[1].str[:-1]
  df.drop(["馬体重(増減)",'性齢'],axis=1, inplace=True)
  # 日付
  df["date"] = pd.to_datetime(df["date"],format="%Y年%m月%d日")

  return df  

## HorseResultsクラス

In [29]:
class HorseResults:
  def __init__(self,horse_vs_df):
    self.horse_results = horse_vs_df[["日付","着順","賞金"]]
    
  def preprocessing(self):
    df = self.horse_results.copy()
    # 着順
    df["着順"] = df["着順"].map(lambda x:str(x).replace('.0', ''))
    # 日付
    df["date"] = pd.to_datetime(df["日付"])
    df.drop(["日付"],axis=1, inplace=True)
    
    self.horse_results = df


In [30]:
hr = HorseResults(horse_vs_df)
hr.preprocessing()
# hr.horse_results
# hr.horse_results.index = hr.horse_results.index.map(lambda x:str(x))

１度目の実行で必ずエラーが出る (なぜか１度目の実行でnanが表示されないから)

In [31]:
results_p = preprocessing(df)
len(results_p)

1366

In [32]:
# pd.options.display.max_rows = None
# for i in df.columns:
#   print(df[i].value_counts())
# results_p.info()

In [33]:
results_p.head()

,着順,枠,馬番,馬名,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,date,race_type,course_ren,weather,ground_state,horse_id,jockey_id,性,齢,体重,体重変化
202101010101,1,5,9,ディーバサンライズ,52.0,△秋山稔,1:08.7,NaN,7,13.5,34.6,4-4,栗東小林,2021-06-12,芝,1200,小雨,良,2018105875,1181,牝,3,474,-22
202101010101,2,3,6,バンベルク,56.0,横山和,1:08.8,3/4,2,3.9,35.0,1-1,栗東安田隆,2021-06-12,芝,1200,小雨,良,2018105258,1140,牡,3,470,-2
202101010101,3,7,14,リツィタル,54.0,大野,1:08.9,1/2,14,124.7,34.0,11-10,栗東牧田,2021-06-12,芝,1200,小雨,良,2018101134,1096,牝,3,394,+2
202101010101,4,5,10,クレマチステソーロ,54.0,鮫島駿,1:09.0,1/2,5,7.6,35.2,2-2,美浦高木,2021-06-12,芝,1200,小雨,良,2018102791,1157,牝,3,468,+2
202101010101,5,3,5,ペイシャケイティー,54.0,古川吉,1:09.0,ハナ,4,7.5,34.5,7-7,美浦土田,2021-06-12,芝,1200,小雨,良,2018101539,1015,牝,3,456,+8


In [34]:
sample_data = results_p["date"].iloc[1]
sample_data

Timestamp('2021-06-12 00:00:00')

sample_dataの日付に走った馬のリスト


In [35]:
pd.options.display.max_rows = None

horse_id_list = results_p[results_p["date"] == sample_data]["horse_id"]
horse_id_list

202101010101    2018105875
202101010101    2018105258
202101010101    2018101134
202101010101    2018102791
202101010101    2018101539
202101010101    2018100570
202101010101    2018102192
202101010101    2018102626
202101010101    2018103582
202101010101    2018101518
202101010101    2018106100
202101010101    2018101447
202101010101    2018109106
202101010101    2018106896
202101010101    2018104780
202101010101    2018100737
202101010102    2018105205
202101010102    2018100974
202101010102    2018104870
202101010102    2018106453
202101010102    2018100926
202101010102    2018102052
202101010102    2018103196
202101010102    2018101898
202101010102    2018102542
202101010102    2018103594
202101010102    2018106609
202101010102    2018100435
202101010102    2018105770
202101010102    2018100706
202101010103    2018100403
202101010103    2018105166
202101010103    2018102691
202101010103    2018100708
202101010103    2018104849
202101010103    2018100066
202101010103    2018103698
2

In [36]:
# hr.horse_results.loc[horse_id_list]

KeyError: '[2018106086, 2018102272, 2018102229, 2017105400, 2018101729, 2015105960] not in index'

In [ ]:
# hr.horse_results

In [ ]:
# horse_vs_df["2018104222"]